# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-07-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-07-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-08 05:22:42,33.93911,67.709953,53207,2253,43291,7663.0,Afghanistan,136.679341,4.234405
1,NaN,NaN,NaN,Albania,2021-01-08 05:22:42,41.15330,20.168300,61705,1223,36535,23947.0,Albania,2144.172632,1.982011
2,NaN,NaN,NaN,Algeria,2021-01-08 05:22:42,28.03390,1.659600,101382,2792,68589,30001.0,Algeria,231.196325,2.753941
3,NaN,NaN,NaN,Andorra,2021-01-08 05:22:42,42.50630,1.521800,8348,84,7615,649.0,Andorra,10804.374555,1.006229
4,NaN,NaN,NaN,Angola,2021-01-08 05:22:42,-11.20270,17.873900,17974,413,11955,5606.0,Angola,54.688290,2.297763


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,51350,51405,51526,51526,51526,51526,53011,53105,53105,53207
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,57146,57727,58316,58316,58991,59438,59623,60283,61008,61705
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,98988,99311,99610,99897,100159,100408,100645,100873,101120,101382


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2179,2181,2191,2191,2191,2191,2237,2244,2244,2253
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1170,1174,1181,1181,1190,1193,1199,1210,1217,1223
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2745,2751,2756,2762,2769,2772,2777,2782,2786,2792


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,41543,41612,41727,41727,41727,41727,42530,42666,42666,43291
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,32700,33185,33634,33634,34353,34648,34996,35551,36102,36535
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,66550,66855,67127,67395,67611,67808,67999,68185,68383,68589


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,4105,4164,4190,4239,4268,4305,4336,4546,4645,4705
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,13172,13392,13601,13823,13955,14064,14187,14440,14656,14845
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1462,1492,1514,1517,1528,1530,1533,1575,1597,1614


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,47,48,48,50,50,50,50,50,50,50
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,156,160,161,169,169,169,169,169,171,171
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,32,32,32,33,33,33,33,33,35,35


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
650,1001.0,Autauga,Alabama,US,2021-01-08 05:22:42,32.539527,-86.644082,4705,50,0,4655.0,"Autauga, Alabama, US",8421.485976,1.062699
687,1075.0,Lamar,Alabama,US,2021-01-08 05:22:42,33.779950,-88.096680,1114,16,0,1098.0,"Lamar, Alabama, US",8069.540022,1.436266
688,1077.0,Lauderdale,Alabama,US,2021-01-08 05:22:42,34.901719,-87.656247,6930,71,0,6859.0,"Lauderdale, Alabama, US",7473.390202,1.024531


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,21574043,365208,0
India,10413417,150570,10037398
Brazil,7961673,200498,7081521


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,21574043,365208,0,21208835,2021-01-08 05:22:42,37.936303,-91.379001
India,10413417,150570,10037398,225449,2021-01-08 05:22:42,23.088275,81.806127
Brazil,7961673,200498,7081521,679654,2021-01-08 05:22:42,-12.669522,-48.480493
Russia,3297833,59628,2681373,556832,2021-01-08 05:22:42,54.546312,62.120860
United Kingdom,2898052,78632,6520,2812900,2021-01-08 05:22:42,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,2570829,28457,0
Texas,1910091,29498,0
Florida,1429722,22481,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,2570829,28457,0,2542372,2021-01-08 05:22:42,37.843962,-120.728594
Texas,1910091,29498,0,1880593,2021-01-08 05:22:42,31.660643,-98.653069
Florida,1429722,22481,0,1407241,2021-01-08 05:22:42,28.940755,-82.700744
New York,1081885,39118,0,1042767,2021-01-08 05:22:42,42.544151,-75.474183
Illinois,1008045,18941,0,989104,2021-01-08 05:22:42,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,871404,11545,0
Illinois,Cook,410005,8630,0
Arizona,Maricopa,360707,5575,0
Florida,Miami-Dade,318115,4297,0
Texas,Harris,253612,3522,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,871404,11545,0,859859,2021-01-08 05:22:42,34.308284,-118.228241,6037.0
Illinois,Cook,410005,8630,0,401375,2021-01-08 05:22:42,41.841448,-87.816588,17031.0
Arizona,Maricopa,360707,5575,0,355132,2021-01-08 05:22:42,33.348359,-112.491815,4013.0
Florida,Miami-Dade,318115,4297,0,313818,2021-01-08 05:22:42,25.611236,-80.551706,12086.0
Texas,Harris,253612,3522,0,250090,2021-01-08 05:22:42,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-05,53105,60283,100873,8308,17756,160,1662730,160544,28536,369721,...,2782709,22104,77295,1,114662,1504,143169,2101,22645,17194
2021-01-06,53105,61008,101120,8348,17864,163,1676171,160853,28546,372190,...,2845265,23048,77350,1,114908,1505,144257,2102,23495,17804
2021-01-07,53207,61705,101382,8348,17974,163,1690006,161054,28571,374730,...,2898052,23807,77412,1,115322,1509,145252,2104,24297,18675


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-05,2244,1210,2782,84,410,5,43785,2878,909,6457,...,76428,217,614,0,1042,35,1505,610,412,418
2021-01-06,2244,1217,2786,84,413,5,43976,2890,909,6525,...,77470,221,615,0,1047,35,1519,610,417,431
2021-01-07,2253,1223,2792,84,413,5,44122,2901,909,6568,...,78632,231,615,0,1052,35,1536,610,423,446


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-05,42666,35551,68185,7585,11376,148,1466616,145759,25802,343039,...,6383,15919,75545,1,108732,1339,124525,1401,19526,11813
2021-01-06,42666,36102,68383,7615,11477,149,1474048,146576,25810,344974,...,6448,16540,75650,1,108966,1353,126110,1403,19672,11966
2021-01-07,43291,36535,68589,7615,11955,149,1484794,147388,25817,346901,...,6520,17098,75759,1,109364,1353,127495,1405,19980,12083


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,4105,4164,4190,4239,4268,4305,4336,4546,4645,4705
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1025,1037,1056,1065,1069,1074,1077,1083,1098,1114
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,6230,6371,6478,6566,6613,6682,6702,6766,6839,6930
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1978,2005,2044,2082,2102,2115,2128,2149,2181,2217
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,10047,10159,10320,10450,10508,10560,10656,10765,10964,11179


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-05,379593,48560,0,567474,238888,2495814,346893,196968,61100,49,...,617649,1864173,288951,8038,2083,371913,258767,93162,531890,45569
2021-01-06,384184,48899,0,574680,242593,2530857,349450,199454,61729,49,...,625380,1884799,292720,8158,2083,377300,260752,94678,535999,45890
2021-01-07,389230,49228,0,584593,245916,2570829,352923,202758,62949,49,...,634237,1910091,297317,8403,2130,382679,264012,96002,540508,46168


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-01-05        4546   14440    1575  1923   4849     920   1574    9975   
2021-01-06        4645   14656    1597  1944   4898     925   1583   10109   
2021-01-07        4705   14845    1614  1981   4957     927   1598   10283   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-05         2474     1454  ...    2136    463     2606      614   
2021-01-06         2519     1468  ...    2168    470     2621      615   
2021-01-07         2552     1496  ...    2169    470     2644      615   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-05           3061  2216  1597          0      783    479  
2021-01-06           3083  2239  1618          0      789    479  
2021-01-07           3108  2260  1668          0      794    483  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-05,4886,218,0,9317,3836,27400,4991,6192,947,0,...,7267,28808,1312,149,24,5191,3541,1442,5366,464
2021-01-06,4994,220,0,9444,3901,27926,5050,6230,947,0,...,7379,29118,1330,152,24,5226,3605,1481,5435,464
2021-01-07,5080,223,0,9741,3926,28457,5102,6287,949,0,...,7492,29498,1359,155,24,5275,3634,1518,5477,489


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-01-05          50     169      33   46     63      22     45     157   
2021-01-06          50     171      35   48     67      22     46     158   
2021-01-07          50     171      35   48     72      22     46     158   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-05           63       22  ...      11      9       17        7   
2021-01-06           63       25  ...      11      9       17        7   
2021-01-07           63       25  ...      15      9       17        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-05             19     4     7          0       21      3  
2021-01-06             19     4     7          0       21      3  
2021-01-07             24     4     8          0       21      4  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-05,0.001773,0.011070,0.002265,0.007152,0.004071,0.00000,0.008363,0.002022,0.000666,0.006290,...,0.022445,0.031644,0.000479,0.0,0.002229,0.004676,0.006616,0.000000,0.029646,0.086234
2021-01-06,0.000000,0.012027,0.002449,0.004815,0.006082,0.01875,0.008084,0.001925,0.000350,0.006678,...,0.022480,0.042707,0.000712,0.0,0.002145,0.000665,0.007599,0.000476,0.037536,0.035477
2021-01-07,0.001921,0.011425,0.002591,0.000000,0.006158,0.00000,0.008254,0.001250,0.000876,0.006824,...,0.018553,0.032931,0.000802,0.0,0.003603,0.002658,0.006897,0.000951,0.034135,0.048922


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-05,0.003129,0.009174,0.001801,0.0,0.004902,0.0,0.003461,0.004888,0.0,0.015731,...,0.011662,0.028436,0.000000,NaN,0.003854,0.0,0.010067,0.0,0.035176,0.088542
2021-01-06,0.000000,0.005785,0.001438,0.0,0.007317,0.0,0.004362,0.004170,0.0,0.010531,...,0.013634,0.018433,0.001629,NaN,0.004798,0.0,0.009302,0.0,0.012136,0.031100
2021-01-07,0.004011,0.004930,0.002154,0.0,0.000000,0.0,0.003320,0.003806,0.0,0.006590,...,0.014999,0.045249,0.000000,NaN,0.004776,0.0,0.011192,0.0,0.014388,0.034803


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-05,0.003198,0.015859,0.002735,0.004902,0.009764,0.000000,0.005852,0.006407,0.000620,0.008129,...,0.019649,0.024455,0.001432,0.0,0.002434,0.000000,0.012596,0.000000,0.015445,0.008279
2021-01-06,0.000000,0.015499,0.002904,0.003955,0.008878,0.006757,0.005067,0.005605,0.000310,0.005641,...,0.010183,0.039010,0.001390,0.0,0.002152,0.010456,0.012728,0.001428,0.007477,0.012952
2021-01-07,0.014649,0.011994,0.003012,0.000000,0.041649,0.000000,0.007290,0.005540,0.000271,0.005586,...,0.011166,0.033736,0.001441,0.0,0.003653,0.000000,0.010982,0.001426,0.015657,0.009778


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-05,0.014697,0.004073,NaN,0.010564,0.017493,0.015343,0.010069,0.011981,0.012713,0.0,...,0.008818,0.018036,0.011616,0.020958,0.013625,0.011909,0.009094,0.013887,0.007614,0.006894
2021-01-06,0.012095,0.006981,NaN,0.012698,0.015509,0.014041,0.007371,0.012621,0.010295,0.0,...,0.012517,0.011064,0.013044,0.014929,0.000000,0.014485,0.007671,0.016273,0.007725,0.007044
2021-01-07,0.013134,0.006728,NaN,0.017250,0.013698,0.015794,0.009938,0.016565,0.019764,0.0,...,0.014163,0.013419,0.015704,0.030032,0.022564,0.014257,0.012502,0.013984,0.008412,0.006058


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-05      0.048432  0.017833  0.027397  0.020159  0.016349  0.010989   
2021-01-06      0.021777  0.014958  0.013968  0.010920  0.010105  0.005435   
2021-01-07      0.012917  0.012896  0.010645  0.019033  0.012046  0.002162   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-05      0.012870  0.022867  0.024431  0.009022  ...  0.010407   
2021-01-06      0.005718  0.013434  0.018189  0.009629  ...  0.014981   
2021-01-07      0.009476  0.017212  0.013100  0.019074  ...  0.000461   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-05      0.019824  0.006178  0.018242   0.002620  0.007731  0.005035   
2021-01-06      0.015119  0.005756  0.001629   0.007187  0.010379  0.013150   
2021-01-07      0.000000  0.008775  0.000000   0.008109  0.009379  0.030902   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-05            NaN -0.001276  0.000000  
2021-01-06            NaN  0.007663  0.000000  
2021-01-07            NaN  0.006337  0.008351  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-05,0.001640,0.000000,NaN,0.027913,0.009474,0.019345,0.009506,0.003891,0.010672,NaN,...,0.013811,0.008330,0.005364,0.034722,0.0,0.011496,0.016944,0.032951,0.018410,0.059361
2021-01-06,0.022104,0.009174,NaN,0.013631,0.016945,0.019197,0.011821,0.006137,0.000000,NaN,...,0.015412,0.010761,0.013720,0.020134,0.0,0.006742,0.018074,0.027046,0.012859,0.000000
2021-01-07,0.017221,0.013636,NaN,0.031449,0.006409,0.019015,0.010297,0.009149,0.002112,NaN,...,0.015314,0.013050,0.021805,0.019737,0.0,0.009376,0.008044,0.024983,0.007728,0.053879


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2021-01-05         0.0  0.000000  0.000000  0.000000  0.000000     0.0   
2021-01-06         0.0  0.011834  0.060606  0.043478  0.063492     0.0   
2021-01-07         0.0  0.000000  0.000000  0.000000  0.074627     0.0   

Province_State                                         ...   Wyoming         \
Admin2            Butler   Calhoun Chambers  Cherokee  ...      Park Platte   
2021-01-05      0.000000  0.000000      0.0  0.000000  ...  0.100000  0.125   
2021-01-06      0.022222  0.006369      0.0  0.136364  ...  0.000000  0.000   
2021-01-07      0.000000  0.000000      0.0  0.000000  ...  0.363636  0.000   

Province_State                                                           \
Admin2         Sheridan  Sublette Sweetwater Teton     Uinta Unassigned   
2021-01-05          0.0  0.166667   0.187500   0.0  0.000000        NaN   
2021-01-06          0.0  0.000000   0.000000   0.0  0.000000        NaN   
2021-01-07          0.0  0.000000   0.263158   0.0  0.142857        NaN   

Province_State                      
Admin2          Washakie    Weston  
2021-01-05      0.105263  0.500000  
2021-01-06      0.000000  0.000000  
2021-01-07      0.000000  0.333333  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-05,0.008157,0.008290,0.002390,0.006590,0.003209,0.000925,0.006377,0.001678,0.000650,0.005413,...,0.022163,0.030169,0.000444,0.0,0.001960,0.004516,0.006968,0.000051,0.023144,0.061858
2021-01-06,0.004078,0.010158,0.002419,0.005703,0.004646,0.009837,0.007230,0.001801,0.000500,0.006046,...,0.022322,0.036438,0.000578,0.0,0.002053,0.002590,0.007284,0.000264,0.030340,0.048668
2021-01-07,0.003000,0.010791,0.002505,0.002851,0.005402,0.004919,0.007742,0.001525,0.000688,0.006435,...,0.020437,0.034685,0.000690,0.0,0.002828,0.002624,0.007091,0.000608,0.032238,0.048795


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-05,0.006924,0.006811,0.001781,0.000049,0.003131,1.490116e-08,0.003132,0.004687,1.075509e-06,0.010902,...,0.008952,0.030565,0.000014,NaN,0.003418,9.349948e-40,0.012719,0.000021,0.021367,0.050033
2021-01-06,0.003462,0.006298,0.001610,0.000025,0.005224,7.450581e-09,0.003747,0.004428,5.377547e-07,0.010716,...,0.011293,0.024499,0.000822,NaN,0.004108,4.674974e-40,0.011011,0.000010,0.016751,0.040567
2021-01-07,0.003736,0.005614,0.001882,0.000012,0.002612,3.725290e-09,0.003534,0.004117,2.688773e-07,0.008653,...,0.013146,0.034874,0.000411,NaN,0.004442,2.337487e-40,0.011101,0.000005,0.015570,0.037685


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-05,0.006507,0.013309,0.002902,0.004685,0.007334,0.000068,0.005278,0.005334,0.000422,0.006601,...,0.026675,0.028776,0.001228,0.0,0.002181,0.000853,0.012199,0.000730,0.011523,0.010249
2021-01-06,0.003253,0.014404,0.002903,0.004320,0.008106,0.003412,0.005173,0.005470,0.000366,0.006121,...,0.018429,0.033893,0.001309,0.0,0.002167,0.005654,0.012464,0.001079,0.009500,0.011600
2021-01-07,0.008951,0.013199,0.002958,0.002160,0.024877,0.001706,0.006231,0.005505,0.000319,0.005853,...,0.014798,0.033815,0.001375,0.0,0.002910,0.002827,0.011723,0.001252,0.012578,0.010689


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-05,0.011038,0.005543,NaN,0.013640,0.012764,0.015364,0.008313,0.013776,0.012806,5.347778e-90,...,0.009082,0.013235,0.010069,0.017540,0.008985,0.011872,0.010338,0.014070,0.005670,0.006838
2021-01-06,0.011566,0.006262,NaN,0.013169,0.014137,0.014702,0.007842,0.013199,0.011550,2.673889e-90,...,0.010799,0.012150,0.011556,0.016234,0.004492,0.013178,0.009005,0.015172,0.006698,0.006941
2021-01-07,0.012350,0.006495,NaN,0.015209,0.013917,0.015248,0.008890,0.014882,0.015657,1.336945e-90,...,0.012481,0.012784,0.013630,0.023133,0.013528,0.013717,0.010753,0.014578,0.007555,0.006500


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-05      0.028173  0.013662  0.015407  0.012750  0.011111  0.011145   
2021-01-06      0.024975  0.014310  0.014688  0.011835  0.010608  0.008290   
2021-01-07      0.018946  0.013603  0.012666  0.015434  0.011327  0.005226   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-05      0.009951  0.013702  0.015701  0.006138  ...  0.012883   
2021-01-06      0.007834  0.013568  0.016945  0.007883  ...  0.013932   
2021-01-07      0.008655  0.015390  0.015023  0.013478  ...  0.007197   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-05      0.015532  0.006151  0.011306   0.005985  0.009720  0.006403   
2021-01-06      0.015325  0.005953  0.006467   0.006586  0.010050  0.009776   
2021-01-07      0.007663  0.007364  0.003234   0.007347  0.009714  0.020339   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-05           -1.0  0.000287  0.001329  
2021-01-06           -1.0  0.003975  0.000665  
2021-01-07           -1.0  0.005156  0.004508  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-05,0.001664,0.006553,NaN,0.015319,0.009771,0.014055,0.007085,0.006045,0.007648,NaN,...,0.014000,0.005876,0.005719,0.027376,0.010870,0.007367,0.010589,0.022007,0.010096,0.031123
2021-01-06,0.011884,0.007864,NaN,0.014475,0.013358,0.016626,0.009453,0.006091,0.003824,NaN,...,0.014706,0.008318,0.009719,0.023755,0.005435,0.007055,0.014332,0.024526,0.011477,0.015562
2021-01-07,0.014552,0.010750,NaN,0.022962,0.009883,0.017820,0.009875,0.007620,0.002968,NaN,...,0.015010,0.010684,0.015762,0.021746,0.002717,0.008216,0.011188,0.024755,0.009602,0.034720


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-05      0.001492  0.001961  0.000977  0.000744  0.000679  0.001563   
2021-01-06      0.000746  0.006898  0.030791  0.022111  0.032086  0.000781   
2021-01-07      0.000373  0.003449  0.015396  0.011056  0.053356  0.000391   

Province_State                                              ...   Wyoming  \
Admin2                Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-05      4.405850e-08  0.000622  0.000381 -0.000340  ...  0.056696   
2021-01-06      1.111113e-02  0.003496  0.000190  0.068012  ...  0.028348   
2021-01-07      5.555567e-03  0.001748  0.000095  0.034006  ...  0.195992   

Province_State                                                     \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton   
2021-01-05      0.062500  0.001107  0.086458   0.094792  0.006185   
2021-01-06      0.031250  0.000553  0.043229   0.047396  0.003092   
2021-01-07      0.015625  0.000277  0.021615   0.155277  0.001546   

Province_State                                               
Admin2                 Uinta Unassigned  Washakie    Weston  
2021-01-05      8.216981e-10  -0.501961  0.056215  0.250000  
2021-01-06      4.108491e-10  -0.501961  0.028108  0.125000  
2021-01-07      7.142857e-02  -0.501961  0.014054  0.229167  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210107,AK,47660,NaN,1278882,NaN,totalTestsViral,1326542,89.0,1084.0,...,1326542,3,10,42c4c918268996d64ce315a1e11e13479ccd1d8b,0,0,0,0,0,NaN
1,20210107,AL,389230,77647.0,1620448,NaN,totalTestsPeopleViral,1932031,3015.0,36411.0,...,2009678,86,352,e62d9507d10048d80a8c8ccf049a81d980549f34,0,0,0,0,0,NaN
2,20210107,AR,245916,45385.0,1928699,NaN,totalTestsViral,2129230,1326.0,11900.0,...,2174615,25,33,f697ab2ae871ae5b13f0308d8082f2f0d59a1dd6,0,0,0,0,0,NaN
3,20210107,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,76d2f8f3df793c9bd8ad2cf0d87985438eb7414e,0,0,0,0,0,NaN
4,20210107,AZ,584593,30620.0,2435839,NaN,totalTestsViral,5416054,4920.0,42112.0,...,3020432,297,1049,d918a4a2e20a2c5b6cf081296906a53e92f89e38,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

/Users/Brian/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210107,AK,47660.0,NaN,1278882.0,NaN,totalTestsViral,1326542.0,89.0,1084.0,...,1326542,3,10,42c4c918268996d64ce315a1e11e13479ccd1d8b,0,0,0,0,0,NaN
1,20210107,AL,389230.0,77647.0,1620448.0,NaN,totalTestsPeopleViral,1932031.0,3015.0,36411.0,...,2009678,86,352,e62d9507d10048d80a8c8ccf049a81d980549f34,0,0,0,0,0,NaN
2,20210107,AR,245916.0,45385.0,1928699.0,NaN,totalTestsViral,2129230.0,1326.0,11900.0,...,2174615,25,33,f697ab2ae871ae5b13f0308d8082f2f0d59a1dd6,0,0,0,0,0,NaN
3,20210107,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,76d2f8f3df793c9bd8ad2cf0d87985438eb7414e,0,0,0,0,0,NaN
4,20210107,AZ,584593.0,30620.0,2435839.0,NaN,totalTestsViral,5416054.0,4920.0,42112.0,...,3020432,297,1049,d918a4a2e20a2c5b6cf081296906a53e92f89e38,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-07,AK,47660,NaN,1278882,NaN,totalTestsViral,1326542,89.0,1084.0,NaN,...,1326542,3,10,42c4c918268996d64ce315a1e11e13479ccd1d8b,0,0,0,0,0,NaN
2021-01-07,AL,389230,77647.0,1620448,NaN,totalTestsPeopleViral,1932031,3015.0,36411.0,NaN,...,2009678,86,352,e62d9507d10048d80a8c8ccf049a81d980549f34,0,0,0,0,0,NaN
2021-01-07,AR,245916,45385.0,1928699,NaN,totalTestsViral,2129230,1326.0,11900.0,427.0,...,2174615,25,33,f697ab2ae871ae5b13f0308d8082f2f0d59a1dd6,0,0,0,0,0,NaN
2021-01-07,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,76d2f8f3df793c9bd8ad2cf0d87985438eb7414e,0,0,0,0,0,NaN
2021-01-07,AZ,584593,30620.0,2435839,NaN,totalTestsViral,5416054,4920.0,42112.0,1101.0,...,3020432,297,1049,d918a4a2e20a2c5b6cf081296906a53e92f89e38,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-07,AK,47660.0,NaN,1278882.0,NaN,totalTestsViral,1326542.0,89.0,1084.0,NaN,...,1326542,3,10,42c4c918268996d64ce315a1e11e13479ccd1d8b,0,0,0,0,0,NaN
2021-01-07,AL,389230.0,77647.0,1620448.0,NaN,totalTestsPeopleViral,1932031.0,3015.0,36411.0,NaN,...,2009678,86,352,e62d9507d10048d80a8c8ccf049a81d980549f34,0,0,0,0,0,NaN
2021-01-07,AR,245916.0,45385.0,1928699.0,NaN,totalTestsViral,2129230.0,1326.0,11900.0,427.0,...,2174615,25,33,f697ab2ae871ae5b13f0308d8082f2f0d59a1dd6,0,0,0,0,0,NaN
2021-01-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,76d2f8f3df793c9bd8ad2cf0d87985438eb7414e,0,0,0,0,0,NaN
2021-01-07,AZ,584593.0,30620.0,2435839.0,NaN,totalTestsViral,5416054.0,4920.0,42112.0,1101.0,...,3020432,297,1049,d918a4a2e20a2c5b6cf081296906a53e92f89e38,0,0,0,0,0,NaN
